# Getting Elevation Info From the Austrian Elevation Service

[![Binder](http://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/ogd-at-lab/main?urlpath=lab/tree/notebooks/elevation.ipynb)

Homepage of the service: https://maegger.github.io/getAustrianElevation.html (Copyright (c) 2017, Manfred Egger)

Related QGIS plugin: https://github.com/maegger/AustrianElevation

Elevation data source: CC BY 3.0 AT http://geoland.at/ 


In [ ]:
import hvplot.pandas
from geopy.geocoders import Nominatim
from utils.dataaccess import get_elevation
from utils.converting import location_to_gdf

In [ ]:
address = "Stephansdom, Wien"
locator = Nominatim(user_agent="OGD.AT-Lab")
location = locator.geocode(address)
print(location.address)
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
gdf = location_to_gdf(location, address)

Before we can query the elevation, we need to reproject the coordinates to EPSG:3857

In [ ]:
gdf = gdf.to_crs('epsg:3857')
gdf

In [ ]:
gdf.loc[0, 'elevation'] = get_elevation(gdf.iloc[0].geometry)
gdf.to_crs('epsg:4326')

## Fetching elevation data for a whole GeoDataFrame

In [ ]:
from utils.dataaccess import get_gdf_from_wfs
citybike_gdf = get_gdf_from_wfs('CITYBIKEOGD')
citybike_gdf = citybike_gdf.to_crs('epsg:3857')

In [ ]:
def get_elevation_for_row(row):
    return get_elevation(row.geometry)

In [ ]:
citybike_gdf['elevation'] = citybike_gdf.apply(get_elevation_for_row, axis=1)

In [ ]:
citybike_gdf = citybike_gdf.to_crs('epsg:4326')

In [ ]:
citybike_gdf.hvplot(c='elevation', geo=True, tiles='OSM', title='Citybike station elevation')